In [ ]:
# import torch
# from torch.utils.data import DataLoader
# from torchvision import transforms
# from PIL import Image
# from diffusers import StableDiffusionImg2ImgPipeline
# from torch.optim import Adam
# from tqdm import tqdm
# from glob import glob
# import os

In [ ]:
!pip install transformers accelerate diffusers
import torch
from diffusers import StableDiffusionImg2ImgPipeline
from transformers import AdamW
from tqdm.auto import tqdm
from PIL import Image
import glob
from torchvision import transforms

In [ ]:
# # 1. Load Dataset
# def load_images(image_paths, transform):
#     images = []
#     for path in image_paths:
#         image = Image.open(path).convert("RGB")
#         images.append(transform(image))
#     return torch.stack(images)

In [ ]:
def load_images(image_paths, transform):
    images = [Image.open(path).convert("RGB") for path in image_paths]
    transformed_images = [transform(image) for image in images]
    return torch.stack(transformed_images)  # Stack images into a batch

In [ ]:
# # Define transformations
# transform = transforms.Compose([
#     transforms.Resize((256, 256)),
#     transforms.ToTensor(),
#     transforms.Normalize([0.5, 0.5, 0.5], [0.5, 0.5, 0.5]),
# ])

In [ ]:
# Preprocessing transform for images
transform = transforms.Compose([
    transforms.Resize((512, 512)),  # Resize images to 512x512
    transforms.ToTensor(),
])

In [ ]:
!unzip '/content/Real.zip'

In [18]:
# Load image paths (replace with your dataset path)
image_paths = glob("/content/Real/*.jpg")  # Update this path
batch_size = 4

TypeError: 'module' object is not callable

In [4]:
from glob import glob  # Import the glob function from the glob module

# Load image paths (replace with your dataset path)
image_paths = glob("/content/Real/*.jpg")  # Update this path
batch_size = 4

In [ ]:
def get_image_extensions(dataset_path):
    extensions = set()
    for root, dirs, files in os.walk(dataset_path):
        for file in files:
            # Get the file extension
            extension = os.path.splitext(file)[1]
            # Add the extension to the set
            extensions.add(extension.lower())
    return extensions

In [ ]:
image_extensions = get_image_extensions(dataset_path)
print("Image extensions in the dataset:")
for extension in image_extensions:
    print(extension)

In [ ]:
dataset_dir = dataset_path

unique_channels = set()

for folder_type in os.listdir(dataset_dir):
    folder_path = os.path.join(dataset_dir, folder_type)

    if not os.path.isdir(folder_path):
        continue

    for label in os.listdir(folder_path):
        label_path = os.path.join(folder_path, label)

        if not os.path.isdir(label_path):
            continue

        for filename in os.listdir(label_path):
            img = mpimg.imread(os.path.join(label_path, filename))
            channels = img.shape[2]
            unique_channels.add(channels)

print("Unique channels:", unique_channels)

In [5]:
print(f"Number of images found: {len(image_paths)}")

Number of images found: 6321


In [ ]:
# # Create DataLoader
# image_batches = [
#     load_images(image_paths[i:i + batch_size], transform)
#     for i in range(0, len(image_paths), batch_size)
# ]

In [ ]:
# Create DataLoader
image_batches = [
    load_images(image_paths[i:i + batch_size], transform)
    for i in range(0, len(image_paths), batch_size)
]

In [ ]:
# model = StableDiffusionImg2ImgPipeline.from_pretrained("CompVis/stable-diffusion-v1-4").to("cpu")
# optimizer = Adam(model.unet.parameters(), lr=5e-5)

Loading pipeline components...:   0%|          | 0/7 [00:00<?, ?it/s]

In [ ]:
# Load the model and optimizer
model = StableDiffusionImg2ImgPipeline.from_pretrained("CompVis/stable-diffusion-v1-4",
                                                      revision="fp16",
                                                      torch_dtype=torch.float16).to("cpu") # Load the model in fp16 precision on the GPU

In [ ]:
## 3. Fine-Tune the Model
epochs = 5
loss_fn = torch.nn.MSELoss()

for epoch in range(epochs):
    print(f"Epoch {epoch + 1}/{epochs}")
    epoch_loss = 0
    for batch_images in tqdm(image_batches):
        batch_images = batch_images.to("cpu")

        # Generate noisy images (input for image-to-image generation)
        noise = torch.randn_like(batch_images).to("cpu")
        noisy_images = batch_images + noise * 0.1

        # Model prediction
        # Provide a generic prompt for fine-tuning
        outputs = model(prompt="an image", image=noisy_images).images
        loss = loss_fn(outputs, batch_images)

        # Backpropagation
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        epoch_loss += loss.item()
    print(f"Epoch Loss: {epoch_loss:.4f}")

Epoch 1/5


  0%|          | 0/1581 [00:00<?, ?it/s]/usr/local/lib/python3.10/dist-packages/diffusers/image_processor.py:704: FutureWarning: Passing `image` as torch tensor with value range in [-1,1] is deprecated. The expected value range for image tensor is [0,1] when passing as pytorch tensor or numpy Array. You passed `image` with value range [-1.4076532125473022,1.2903332710266113]
  warnings.warn(


  0%|          | 0/40 [00:00<?, ?it/s]

  0%|          | 0/1581 [00:31<?, ?it/s]


RuntimeError: The size of tensor a (4096) must match the size of tensor b (1024) at non-singleton dimension 1

In [ ]:
# 4. Test the Model
test_image_path = "path_to_test_image.jpg"  # Replace with your test image path
test_image = Image.open(test_image_path).convert("RGB")
test_image_tensor = transform(test_image).unsqueeze(0).to("cuda")